In [1]:
import os
import csv
import numpy as np

import torch

from dataset import Dataset
from feature_extractor import FeatureExtractor
from evaluation import get_auroc, get_average_precision, get_tnr_frac_tpr
from monitors import OutsideTheBoxMonitor, GaussianMixtureMonitor, MaxSoftmaxProbabilityMonitor, MaxLogitMonitor, \
    EnergyMonitor, ReActMonitor, MahalanobisMonitor

from sklearn.mixture import GaussianMixture
from sklearn.metrics import average_precision_score

In [8]:
batch_size = 10
device_name = 'cuda:0' if torch.cuda.is_available() else 'cpu'

# id_dataset = "cifar10"
model = "densenet"
layer = 98

# novelties = ["cifar100", "svhn", "lsun"]

all_id_datasets = ["cifar10", "svhn", "cifar100"]

all_ood_datasets = [["cifar100", "svhn", "lsun"],
                    ["cifar10", "tiny_imagenet", "lsun"],
                    ["cifar10", "svhn", "lsun"]]

perturbations = ["brightness", "blur", "pixelization"]
attacks = ["fgsm", "deepfool", "pgd"]

cov_constraints = ["full"]#, "diag", "tied", "spherical"]
n_clusters = [1, 2, 3, 5]
is_novel = [1, 1, 1, 0, 0, 0, 0, 0, 0]

In [9]:
def select_parameters_gmm(features_train, pred_train, lab_train,
                          features_test, pred_test, lab_test,
                          features_ood, pred_ood, lab_ood,
                          is_novel_ood, eval_idx,
                          cov_constraints_list, n_clusters_list, 
                          selection_metric = "aupr"):
    
    print("Tuning GMM hyperparameters for dataset ", eval_idx)
    
    correct_indices_train = (lab_train == pred_train)
    
    features_train = features_train[correct_indices_train]
    lab_train = lab_train[correct_indices_train]
    pred_train = pred_train[correct_indices_train]
    
    N = features_test.shape[0] // 2
    features_test_selec = features_test[N:]
    pred_test_selec = pred_test[N:]
    lab_test_selec = lab_test[N:]

    features_test_eval = features_test[:N]
    pred_test_eval = pred_test[:N]
    lab_test_eval = lab_test[:N]
    
    n_ood_datasets = len(lab_ood)
    select_indices = [k for k in range(n_ood_datasets) if k is not eval_idx]
    
    features_ood_selec = [features_ood[i][:N] for i in select_indices]
    pred_ood_selec = [pred_ood[i][:N] for i in select_indices]
    lab_ood_selec = [lab_ood[i][:N] for i in select_indices]
    is_novel_ood_selec = [is_novel_ood[i] for i in select_indices]
        
    features_ood_eval = features_ood[eval_idx][:N]
    pred_ood_eval = pred_ood[eval_idx][:N]
    lab_ood_eval = lab_ood[eval_idx][:N]
    is_novel_ood_eval = is_novel_ood[eval_idx]
    
    if selection_metric == "aupr":
        metric = get_average_precision
    elif selection_metric == "auroc":
        metric = get_auroc
    elif selection_metric == "tnr95tpr":
        metric = get_tnr_frac_tpr
    else:
        "Error, unsupported selection metric. Valid choices are 'aupr', 'auroc' and 'tnr95tpr'"
        
    constraints, n_components = [], []
    for cat in list(set(lab_train)):
        print("... Selecting hyperparameters for class ", cat)
        
        f_train = features_train[pred_train == cat]
        
        res_config = []
        for cc in  cov_constraints:
            for n in n_clusters:
                gmm = GaussianMixture(n_components=n, covariance_type=cc)
                gmm.fit(f_train)
                
                f = np.concatenate([features_test_selec[pred_test_selec == cat], 
                                    np.concatenate([features_ood_selec[i][pred_ood_selec[i] == cat] for i in range(len(features_ood_selec))])])
                p = np.concatenate([pred_test_selec[pred_test_selec == cat], 
                                    np.concatenate([pred_ood_selec[i][pred_ood_selec[i] == cat] for i in range(len(features_ood_selec))])])
                temp = []
                for i in range(len(features_ood_selec)):
                    if is_novel_ood_selec[i]:
                        temp.append(-np.ones(lab_ood_selec[i][pred_ood_selec[i] == cat].shape))
                    else:
                        temp.append(lab_ood_selec[i][pred_ood_selec[i] == cat])
                l = np.concatenate([lab_test_selec[pred_test_selec == cat], 
                                    np.concatenate(temp)])
                try:
                    scores = -gmm.score_samples(f)
                except:
                    pass
                    
                res = metric(scores, l, p)
#                     print(cc, n, res[-1])
                
                res_config.append(res)
        res_config = np.array(res_config)

#         ranks = np.zeros(res_config.shape)
#         for i in range(res_config.shape[1]):
#             temp = res_config[:, i].argsort()
#             ranks[:, i] = np.empty_like(temp)
#             ranks[temp, i] = np.arange(len(res_config))
#         avg_ranks = np.mean(ranks, axis = 1)

        constraints.append(cov_constraints[np.argmax(res_config) // len(n_clusters)])
        n_components.append(n_clusters[np.argmax(res_config) % len(n_clusters)])
        
#         print(constraints[-1], n_components[-1], "\n")
        
    f_eval = np.concatenate([features_test_eval, features_ood_eval])
    p_eval = np.concatenate([pred_test_eval, pred_ood_eval])
    if is_novel_ood_eval:
        l_eval = np.concatenate([lab_test_eval, -np.ones(lab_ood_eval.shape)])
    else:
        l_eval = np.concatenate([lab_test_eval, lab_ood_eval])
        
    monitor = GaussianMixtureMonitor(id_dataset, model, layer, n_components=n_components, constraint=constraints)
    monitor.fit(features_train, pred_train, lab_train, save=False)
    
    scores = -monitor.predict(f_eval, p_eval)
    
    ap = get_average_precision(scores, l_eval, p_eval)
    auroc = get_auroc(scores, l_eval, p_eval)
    tnr = get_tnr_frac_tpr(scores, l_eval, p_eval)
    
    print("AP       = ", ap)
    print("AUROC    = ", auroc)
    print("TNR95TPR = ", tnr)
    
    return constraints, n_components, ap, auroc, tnr

In [10]:
for i_id, id_dataset in enumerate(all_id_datasets):
    print("ID dataset: ", id_dataset)
    
    train_dataset = Dataset(id_dataset, "train", model, batch_size=batch_size)
    test_dataset = Dataset(id_dataset, "test", model, batch_size=batch_size)
    
    feature_extractor = FeatureExtractor(model, id_dataset, [layer], device_name)
    
    novelties = all_ood_datasets[i_id]
    
    ood_names = [novelties[0], novelties[1], novelties[2], 
                 perturbations[0], perturbations[1], perturbations[2],
                 attacks[0], attacks[1], attacks[2]]
    
    ood_datasets = []
    ood_datasets.append(Dataset(novelties[0], "test", model, batch_size=batch_size))
    ood_datasets.append(Dataset(novelties[1], "test", model, batch_size=batch_size))
    ood_datasets.append(Dataset(novelties[2], "test", model, batch_size=batch_size))
    ood_datasets.append(Dataset(id_dataset, "test", model, additional_transform=perturbations[0], batch_size=batch_size))
    ood_datasets.append(Dataset(id_dataset, "test", model, additional_transform=perturbations[1], batch_size=batch_size))
    ood_datasets.append(Dataset(id_dataset, "test", model, additional_transform=perturbations[2], batch_size=batch_size))
    ood_datasets.append(Dataset(id_dataset, "test", model, adversarial_attack=attacks[0], batch_size=batch_size))
    ood_datasets.append(Dataset(id_dataset, "test", model, adversarial_attack=attacks[1], batch_size=batch_size))
    ood_datasets.append(Dataset(id_dataset, "test", model, adversarial_attack=attacks[2], batch_size=batch_size))

    features_train, logits_train, softmax_train, pred_train, lab_train = feature_extractor.get_features(train_dataset)
    features_train = features_train[0]
    features_test, logits_test, softmax_test, pred_test, lab_test = feature_extractor.get_features(test_dataset)
    features_test = features_test[0]

    features_ood, logits_ood, softmax_ood, pred_ood, lab_ood = [], [], [], [], []
    for k in range(len(ood_datasets)):
        features, logits, softmax, pred, lab = feature_extractor.get_features(ood_datasets[k])
        features_ood.append(features[0])
        logits_ood.append(logits)
        softmax_ood.append(softmax)
        pred_ood.append(pred)
        lab_ood.append(lab)

    for eval_idx in range(9):
        print("... OOD dataset: ", ood_names[eval_idx])
        constraints, n_components, ap, auroc, tnr = select_parameters_gmm(features_train, pred_train, lab_train,
                                                                          features_test, pred_test, lab_test,
                                                                          features_ood, pred_ood, lab_ood,
                                                                          is_novel, eval_idx,
                                                                          cov_constraints, n_clusters, 
                                                                          selection_metric = "tnr95tpr")
        print(constraints, n_components, ap, auroc, tnr)
        print()

ID dataset:  cifar10
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Using downloaded and verified file: ./Data/test_32x32.mat
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
... OOD dataset:  cifar100
Tuning GMM hyperparameters for dataset  0
... Selecting hyperparameters for class  0
... Selecting hyperparameters for class  1
... Selecting hyperparameters for class  2
... Selecting hyperparameters for class  3
... Selecting hyperparameters for class  4
... Selecting hyperparameters for class  5
... Selecting hyperparameters for class  6
... Selecting hyperparameters for class  7
... Selecting hyperparameters for class  8
... Selecting hyperparameters for class  9
AP       =  0.8603478620470574
AUROC    =  0.861846335839599
TNR95TPR =  0.50357

/home/joris/Softwares/miniconda3/envs/neural-network-monitoring-benchmark/lib/python3.10/site-packages/sklearn/mixture/_base.py:286: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  warnings.warn(


... Selecting hyperparameters for class  1
... Selecting hyperparameters for class  2
... Selecting hyperparameters for class  3
... Selecting hyperparameters for class  4
... Selecting hyperparameters for class  5
... Selecting hyperparameters for class  6
... Selecting hyperparameters for class  7
... Selecting hyperparameters for class  8
... Selecting hyperparameters for class  9


/home/joris/Softwares/miniconda3/envs/neural-network-monitoring-benchmark/lib/python3.10/site-packages/sklearn/mixture/_base.py:286: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  warnings.warn(


AP       =  0.8166230512833647
AUROC    =  0.844633412125648
TNR95TPR =  0.5111827523675196
['full', 'full', 'full', 'full', 'full', 'full', 'full', 'full', 'full', 'full'] [5, 1, 3, 5, 2, 5, 5, 1, 3, 1] 0.8166230512833647 0.844633412125648 0.5111827523675196

ID dataset:  svhn
Using downloaded and verified file: ./Data/train_32x32.mat
Using downloaded and verified file: ./Data/test_32x32.mat
Files already downloaded and verified
Using downloaded and verified file: ./Data/test_32x32.mat
Using downloaded and verified file: ./Data/test_32x32.mat
Using downloaded and verified file: ./Data/test_32x32.mat
Using downloaded and verified file: ./Data/test_32x32.mat
Using downloaded and verified file: ./Data/test_32x32.mat
Using downloaded and verified file: ./Data/test_32x32.mat
... OOD dataset:  cifar10
Tuning GMM hyperparameters for dataset  0
... Selecting hyperparameters for class  0
... Selecting hyperparameters for class  1
... Selecting hyperparameters for class  2
... Selecting hyperpa

...

... Selecting hyperparameters for class  2
... Selecting hyperparameters for class  3
... Selecting hyperparameters for class  4
... Selecting hyperparameters for class  5
... Selecting hyperparameters for class  6
... Selecting hyperparameters for class  7
... Selecting hyperparameters for class  8
... Selecting hyperparameters for class  9
... Selecting hyperparameters for class  10
... Selecting hyperparameters for class  11
... Selecting hyperparameters for class  12
... Selecting hyperparameters for class  13
... Selecting hyperparameters for class  14
... Selecting hyperparameters for class  15
... Selecting hyperparameters for class  16
... Selecting hyperparameters for class  17
... Selecting hyperparameters for class  18
... Selecting hyperparameters for class  19
... Selecting hyperparameters for class  20
... Selecting hyperparameters for class  21
... Selecting hyperparameters for class  22
... Selecting hyperparameters for class  23
... Selecting hyperparameters for class 

... Selecting hyperparameters for class  61
... Selecting hyperparameters for class  62
... Selecting hyperparameters for class  63
... Selecting hyperparameters for class  64
... Selecting hyperparameters for class  65
... Selecting hyperparameters for class  66
... Selecting hyperparameters for class  67
... Selecting hyperparameters for class  68
... Selecting hyperparameters for class  69
... Selecting hyperparameters for class  70
... Selecting hyperparameters for class  71
... Selecting hyperparameters for class  72
... Selecting hyperparameters for class  73
... Selecting hyperparameters for class  74
... Selecting hyperparameters for class  75
... Selecting hyperparameters for class  76
... Selecting hyperparameters for class  77
... Selecting hyperparameters for class  78
... Selecting hyperparameters for class  79
... Selecting hyperparameters for class  80
... Selecting hyperparameters for class  81
... Selecting hyperparameters for class  82
... Selecting hyperparameters fo

... Selecting hyperparameters for class  2
... Selecting hyperparameters for class  3
... Selecting hyperparameters for class  4
... Selecting hyperparameters for class  5
... Selecting hyperparameters for class  6
... Selecting hyperparameters for class  7
... Selecting hyperparameters for class  8
... Selecting hyperparameters for class  9
... Selecting hyperparameters for class  10
... Selecting hyperparameters for class  11
... Selecting hyperparameters for class  12
... Selecting hyperparameters for class  13
... Selecting hyperparameters for class  14
... Selecting hyperparameters for class  15
... Selecting hyperparameters for class  16
... Selecting hyperparameters for class  17
... Selecting hyperparameters for class  18
... Selecting hyperparameters for class  19
... Selecting hyperparameters for class  20
... Selecting hyperparameters for class  21
... Selecting hyperparameters for class  22
... Selecting hyperparameters for class  23
... Selecting hyperparameters for class 

... Selecting hyperparameters for class  59
... Selecting hyperparameters for class  60
... Selecting hyperparameters for class  61
... Selecting hyperparameters for class  62
... Selecting hyperparameters for class  63
... Selecting hyperparameters for class  64
... Selecting hyperparameters for class  65
... Selecting hyperparameters for class  66
... Selecting hyperparameters for class  67
... Selecting hyperparameters for class  68
... Selecting hyperparameters for class  69
... Selecting hyperparameters for class  70
... Selecting hyperparameters for class  71
... Selecting hyperparameters for class  72
... Selecting hyperparameters for class  73
... Selecting hyperparameters for class  74
... Selecting hyperparameters for class  75
... Selecting hyperparameters for class  76
... Selecting hyperparameters for class  77
... Selecting hyperparameters for class  78
... Selecting hyperparameters for class  79
... Selecting hyperparameters for class  80
... Selecting hyperparameters fo

... Selecting hyperparameters for class  2
... Selecting hyperparameters for class  3
... Selecting hyperparameters for class  4
... Selecting hyperparameters for class  5
... Selecting hyperparameters for class  6
... Selecting hyperparameters for class  7
... Selecting hyperparameters for class  8
... Selecting hyperparameters for class  9
... Selecting hyperparameters for class  10
... Selecting hyperparameters for class  11
... Selecting hyperparameters for class  12
... Selecting hyperparameters for class  13
... Selecting hyperparameters for class  14
... Selecting hyperparameters for class  15
... Selecting hyperparameters for class  16
... Selecting hyperparameters for class  17
... Selecting hyperparameters for class  18
... Selecting hyperparameters for class  19
... Selecting hyperparameters for class  20
... Selecting hyperparameters for class  21
... Selecting hyperparameters for class  22
... Selecting hyperparameters for class  23
... Selecting hyperparameters for class 

In [23]:
def select_parameters_gmm(features_train, pred_train, lab_train,
                          features_test, pred_test, lab_test,
                          features_ood, pred_ood, lab_ood,
                          is_novel_ood, eval_idx,
                          cov_constraints_list, n_clusters_list, 
                          selection_metric = "aupr"):
    
    print("Tuning GMM hyperparameters for dataset ", eval_idx)
    
    correct_indices_train = (lab_train == pred_train)
    
    features_train = features_train[correct_indices_train]
    lab_train = lab_train[correct_indices_train]
    pred_train = pred_train[correct_indices_train]
    
    N = features_test.shape[0] // 2
    features_test_selec = features_test[N:]
    pred_test_selec = pred_test[N:]
    lab_test_selec = lab_test[N:]

    features_test_eval = features_test[:N]
    pred_test_eval = pred_test[:N]
    lab_test_eval = lab_test[:N]
    
    n_ood_datasets = len(lab_ood)
    select_indices = [k for k in range(n_ood_datasets) if k is not eval_idx]
    
    features_ood_selec = [features_ood[i][:N] for i in select_indices]
    pred_ood_selec = [pred_ood[i][:N] for i in select_indices]
    lab_ood_selec = [lab_ood[i][:N] for i in select_indices]
    is_novel_ood_selec = [is_novel_ood[i] for i in select_indices]
        
    features_ood_eval = features_ood[eval_idx][:N]
    pred_ood_eval = pred_ood[eval_idx][:N]
    lab_ood_eval = lab_ood[eval_idx][:N]
    is_novel_ood_eval = is_novel_ood[eval_idx]
    
    if selection_metric == "aupr":
        metric = get_average_precision
    elif selection_metric == "auroc":
        metric = get_auroc
    elif selection_metric == "tnr95tpr":
        metric = get_tnr_frac_tpr
    else:
        "Error, unsupported selection metric. Valid choices are 'aupr', 'auroc' and 'tnr95tpr'"
        
    res_config = []
    for cc in  cov_constraints:
        for n in n_clusters:
            mon = GaussianMixtureMonitor(id_dataset, model, layer, n_components=n, constraint=cc)
            mon.fit(features_train, pred_train, lab_train, save=False)

            f = np.concatenate([features_test_selec, 
                                np.concatenate([features_ood_selec[i] for i in range(len(features_ood_selec))])])
            p = np.concatenate([pred_test_selec, 
                                np.concatenate([pred_ood_selec[i] for i in range(len(features_ood_selec))])])
            temp = []
            for i in range(len(features_ood_selec)):
                if is_novel_ood_selec[i]:
                    temp.append(-np.ones(lab_ood_selec[i].shape))
                else:
                    temp.append(lab_ood_selec[i])
            l = np.concatenate([lab_test_selec, 
                                np.concatenate(temp)])
            try:
                scores = -mon.predict(f, p)
            except:
                pass

            res = metric(scores, l, p)
#                     print(cc, n, res[-1])

            res_config.append(res)
    res_config = np.array(res_config)

#         ranks = np.zeros(res_config.shape)
#         for i in range(res_config.shape[1]):
#             temp = res_config[:, i].argsort()
#             ranks[:, i] = np.empty_like(temp)
#             ranks[temp, i] = np.arange(len(res_config))
#         avg_ranks = np.mean(ranks, axis = 1)

    constraints = cov_constraints[np.argmax(res_config) // len(n_clusters)]
    n_components = n_clusters[np.argmax(res_config) % len(n_clusters)]
        
#         print(constraints[-1], n_components[-1], "\n")
        
    f_eval = np.concatenate([features_test_eval, features_ood_eval])
    p_eval = np.concatenate([pred_test_eval, pred_ood_eval])
    if is_novel_ood_eval:
        l_eval = np.concatenate([lab_test_eval, -np.ones(lab_ood_eval.shape)])
    else:
        l_eval = np.concatenate([lab_test_eval, lab_ood_eval])
        
    monitor = GaussianMixtureMonitor(id_dataset, model, layer, n_components=n_components, constraint=constraints)
    monitor.fit(features_train, pred_train, lab_train, save=False)
    
    scores = -monitor.predict(f_eval, p_eval)
    
    ap = get_average_precision(scores, l_eval, p_eval)
    auroc = get_auroc(scores, l_eval, p_eval)
    tnr = get_tnr_frac_tpr(scores, l_eval, p_eval)
    
    print("AP       = ", ap)
    print("AUROC    = ", auroc)
    print("TNR95TPR = ", tnr)
    
    return constraints, n_components, ap, auroc, tnr

In [11]:
res_config

NameError: name 'res_config' is not defined

In [56]:
def eval_mahalanobis(features_train, pred_train, lab_train,
                     features_test, pred_test, lab_test,
                     features_ood, pred_ood, lab_ood,
                     is_novel_ood, eval_idx):
    
    correct_indices_train = (lab_train == pred_train)
    
    features_train = features_train[correct_indices_train]
    lab_train = lab_train[correct_indices_train]
    pred_train = pred_train[correct_indices_train]
    
    N = features_test.shape[0] // 2

    features_test_eval = features_test[:N]
    pred_test_eval = pred_test[:N]
    lab_test_eval = lab_test[:N]
    
    features_ood_eval = features_ood[eval_idx][:N]
    pred_ood_eval = pred_ood[eval_idx][:N]
    lab_ood_eval = lab_ood[eval_idx][:N]
    is_novel_ood_eval = is_novel_ood[eval_idx]
          
    f_eval = np.concatenate([features_test_eval, features_ood_eval])
    p_eval = np.concatenate([pred_test_eval, pred_ood_eval])
    if is_novel_ood_eval:
        l_eval = np.concatenate([lab_test_eval, -np.ones(lab_ood_eval.shape)])
    else:
        l_eval = np.concatenate([lab_test_eval, lab_ood_eval])
        
    monitor = MahalanobisMonitor(id_dataset, model, layer, is_tied=True)
    monitor.fit(features_train, pred_train, lab_train, save=False)
    
    scores = -monitor.predict(f_eval, p_eval)
    
    ap = get_average_precision(scores, l_eval, p_eval)
    auroc = get_auroc(scores, l_eval, p_eval)
    tnr = get_tnr_frac_tpr(scores, l_eval, p_eval)
    
    print(ap, ",", auroc, ",", tnr)

In [57]:
for i_id, id_dataset in enumerate(all_id_datasets):
    print("ID dataset: ", id_dataset)
    
    train_dataset = Dataset(id_dataset, "train", model, batch_size=batch_size)
    test_dataset = Dataset(id_dataset, "test", model, batch_size=batch_size)
    
    feature_extractor = FeatureExtractor(model, id_dataset, [layer], device_name)
    
    novelties = all_ood_datasets[i_id]
    
    ood_names = [novelties[0], novelties[1], novelties[2], 
                 perturbations[0], perturbations[1], perturbations[2],
                 attacks[0], attacks[1], attacks[2]]
    
    ood_datasets = []
    ood_datasets.append(Dataset(novelties[0], "test", model, batch_size=batch_size))
    ood_datasets.append(Dataset(novelties[1], "test", model, batch_size=batch_size))
    ood_datasets.append(Dataset(novelties[2], "test", model, batch_size=batch_size))
    ood_datasets.append(Dataset(id_dataset, "test", model, additional_transform=perturbations[0], batch_size=batch_size))
    ood_datasets.append(Dataset(id_dataset, "test", model, additional_transform=perturbations[1], batch_size=batch_size))
    ood_datasets.append(Dataset(id_dataset, "test", model, additional_transform=perturbations[2], batch_size=batch_size))
    ood_datasets.append(Dataset(id_dataset, "test", model, adversarial_attack=attacks[0], batch_size=batch_size))
    ood_datasets.append(Dataset(id_dataset, "test", model, adversarial_attack=attacks[1], batch_size=batch_size))
    ood_datasets.append(Dataset(id_dataset, "test", model, adversarial_attack=attacks[2], batch_size=batch_size))

    features_train, logits_train, softmax_train, pred_train, lab_train = feature_extractor.get_features(train_dataset)
    features_train = features_train[0]
    features_test, logits_test, softmax_test, pred_test, lab_test = feature_extractor.get_features(test_dataset)
    features_test = features_test[0]

    features_ood, logits_ood, softmax_ood, pred_ood, lab_ood = [], [], [], [], []
    for k in range(len(ood_datasets)):
        features, logits, softmax, pred, lab = feature_extractor.get_features(ood_datasets[k])
        features_ood.append(features[0])
        logits_ood.append(logits)
        softmax_ood.append(softmax)
        pred_ood.append(pred)
        lab_ood.append(lab)

    for eval_idx in range(9):
#         print("... OOD dataset: ", ood_names[eval_idx])
        eval_mahalanobis(features_train, pred_train, lab_train,
                         features_test, pred_test, lab_test,
                         features_ood, pred_ood, lab_ood,
                         is_novel, eval_idx)
        print()

ID dataset:  cifar10
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Using downloaded and verified file: ./Data/test_32x32.mat
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
0.8912978883573618 , 0.8960762833225303 , 0.6003006872852233
0.9336824531068371 , 0.949204281514291 , 0.8460051546391752
0.97204836974037 , 0.9716006533968148 , 0.8788659793814433
0.5266506343708129 , 0.8213737365366776 , 0.5290869916509561
0.8723534367248703 , 0.926258110035275 , 0.81399665862261
0.6082145179987741 , 0.8380736261207816 , 0.5937231298366294
0.5729919557805294 , 0.8473621477924349 , 0.5925876191755068
0.5653346424763852 , 0.7853443825745168 , 0.5347286821705426
0.6890623699651643 , 0.7911385078346626 , 0.4974571482388397
ID dataset:  svhn
Using downloaded a

In [29]:
def eval_MSP(softmax_test, pred_test, lab_test,
             softmax_ood, pred_ood, lab_ood,
             is_novel_ood, eval_idx):
    
    N = softmax_test.shape[0] // 2

    softmax_test_eval = softmax_test#[:N]
    pred_test_eval = pred_test#[:N]
    lab_test_eval = lab_test#[:N]
    
    softmax_ood_eval = softmax_ood[eval_idx]#[:N]
    pred_ood_eval = pred_ood[eval_idx]#[:N]
    lab_ood_eval = lab_ood[eval_idx]#[:N]
    is_novel_ood_eval = is_novel_ood[eval_idx]
          
    s_eval = np.concatenate([softmax_test_eval, softmax_ood_eval])
    p_eval = np.concatenate([pred_test_eval, pred_ood_eval])
    if is_novel_ood_eval:
        l_eval = np.concatenate([lab_test_eval, -np.ones(lab_ood_eval.shape)])
    else:
        l_eval = np.concatenate([lab_test_eval, lab_ood_eval])
        
    monitor = MaxSoftmaxProbabilityMonitor()
    monitor.fit()
    
    scores = -monitor.predict(s_eval)
    
    ap = get_average_precision(scores, l_eval, p_eval)
    auroc = get_auroc(scores, l_eval, p_eval)
    tnr = get_tnr_frac_tpr(scores, l_eval, p_eval)
    
    print(ap, ",", auroc, ",", tnr)

In [30]:
for i_id, id_dataset in enumerate(all_id_datasets):
    print("ID dataset: ", id_dataset)
    
    train_dataset = Dataset(id_dataset, "train", model, batch_size=batch_size)
    test_dataset = Dataset(id_dataset, "test", model, batch_size=batch_size)
    
    feature_extractor = FeatureExtractor(model, id_dataset, [layer], device_name)
    
    novelties = all_ood_datasets[i_id]
    
    ood_names = [novelties[0], novelties[1], novelties[2], 
                 perturbations[0], perturbations[1], perturbations[2],
                 attacks[0], attacks[1], attacks[2]]
    
    ood_datasets = []
    ood_datasets.append(Dataset(novelties[0], "test", model, batch_size=batch_size))
    ood_datasets.append(Dataset(novelties[1], "test", model, batch_size=batch_size))
    ood_datasets.append(Dataset(novelties[2], "test", model, batch_size=batch_size))
    ood_datasets.append(Dataset(id_dataset, "test", model, additional_transform=perturbations[0], batch_size=batch_size))
    ood_datasets.append(Dataset(id_dataset, "test", model, additional_transform=perturbations[1], batch_size=batch_size))
    ood_datasets.append(Dataset(id_dataset, "test", model, additional_transform=perturbations[2], batch_size=batch_size))
    ood_datasets.append(Dataset(id_dataset, "test", model, adversarial_attack=attacks[0], batch_size=batch_size))
    ood_datasets.append(Dataset(id_dataset, "test", model, adversarial_attack=attacks[1], batch_size=batch_size))
    ood_datasets.append(Dataset(id_dataset, "test", model, adversarial_attack=attacks[2], batch_size=batch_size))

    features_train, logits_train, softmax_train, pred_train, lab_train = feature_extractor.get_features(train_dataset)
    features_train = features_train[0]
    features_test, logits_test, softmax_test, pred_test, lab_test = feature_extractor.get_features(test_dataset)
    features_test = features_test[0]

    features_ood, logits_ood, softmax_ood, pred_ood, lab_ood = [], [], [], [], []
    for k in range(len(ood_datasets)):
        features, logits, softmax, pred, lab = feature_extractor.get_features(ood_datasets[k])
        features_ood.append(features[0])
        logits_ood.append(logits)
        softmax_ood.append(softmax)
        pred_ood.append(pred)
        lab_ood.append(lab)

    for eval_idx in range(9):
#         print("... OOD dataset: ", ood_names[eval_idx])
        eval_MSP(softmax_test, pred_test, lab_test,
                 softmax_ood, pred_ood, lab_ood,
                 is_novel, eval_idx)
#         print()

ID dataset:  cifar10
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Using downloaded and verified file: ./Data/test_32x32.mat
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
0.8975854520060031 , 0.8962354317892822 , 0.5533255044304473
0.963946856489525 , 0.9303592904309907 , 0.7866979822782108
0.9342651366837522 , 0.936813403257378 , 0.7593679940215651
0.6458659526435437 , 0.8699051491761502 , 0.5723304231027535
0.8065605899484907 , 0.8901678282847826 , 0.7461446116908301
0.6673393668430831 , 0.8586827929395987 , 0.5821834560663472
0.6419893598983523 , 0.8710170222006536 , 0.6044691242389777
0.4423443396881372 , 0.7009488223411557 , 0.5525177633611369
0.5541786063723964 , 0.6161407255297326 , 0.16615008922701235
ID dataset:  svhn
Using downloa